In [40]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

In [41]:
def load_mbp10_data(file_path):
    df = pd.read_csv(file_path, parse_dates=['ts_event'])
    price_cols = [col for col in df.columns if 'px' in col]
    for col in price_cols:
        df[col] = df['price'] / 1e9 if col == 'price' else df[col] / 1e9
    return df

In [42]:

def aggregate_to_periods(df, num_periods=390):
    period_length = (df['ts_event'].max() - df['ts_event'].min()).total_seconds() / num_periods
    df['period'] = ((df['ts_event'] - df['ts_event'].min()).dt.total_seconds() // period_length).astype(int)
    agg_cols = [f'bid_px_{i:02d}' for i in range(10)] + [f'ask_px_{i:02d}' for i in range(10)] + \
               [f'bid_sz_{i:02d}' for i in range(10)] + [f'ask_sz_{i:02d}' for i in range(10)]
    period_data = df.groupby('period')[agg_cols].last().reset_index()
    return period_data

In [43]:
def extract_min_ask_bid_with_size(df, X):
    results = []

    for idx, row in df.iterrows():
        period = row['period']
        
        ask_prices = row[[f'ask_px_{i:02}' for i in range(10)]]
        bid_prices = row[[f'bid_px_{i:02}' for i in range(10)]]
        min_ask_idx = ask_prices.idxmin()
        min_ask_price = ask_prices[min_ask_idx]
        position = int(min_ask_idx.split('_')[-1])
        corresponding_bid_price = bid_prices[f'bid_px_{position:02}']
        x_row = X.loc[idx]
        ask_size = x_row[f'ask_sz_{position:02}']
        
        results.append({
            'period': period,
            'min_ask_price': min_ask_price,
            'corresponding_bid_price': corresponding_bid_price,
            'corresponding_ask_size': ask_size
        })
    
    return pd.DataFrame(results)

In [44]:
import pandas as pd

def process_market_data(df):
    df['mid'] = (df['min_ask_price'] + df['corresponding_bid_price']) / 2

    df['impact'] = df['min_ask_price'] - df['mid']

    result_df = df[['period', 'min_ask_price', 'corresponding_bid_price', 'impact']].copy()
    result_df.columns = ['period', 'ask', 'bid', 'impact']
    return result_df

In [45]:
file_path = '/kaggle/input/excels/FROG_2025-04-07 00_00_0000_00.csv'
data = load_mbp10_data(file_path)
period_data = aggregate_to_periods(data)
result_df = extract_min_ask_bid_with_size(period_data, data)
ans = process_market_data(result_df)

In [46]:
ans

,period,ask,bid,impact
0,0.0,2.771000e-08,2.719000e-08,2.600000e-10
1,1.0,2.771000e-08,2.719000e-08,2.600000e-10
2,2.0,2.771000e-08,2.719000e-08,2.600000e-10
3,3.0,2.839000e-08,2.775000e-08,3.200000e-10
4,4.0,2.802000e-08,2.741000e-08,3.050000e-10
...,...,...,...,...
386,386.0,2.851000e-08,2.849000e-08,1.000000e-11
387,387.0,2.848000e-08,2.844000e-08,2.000000e-11
388,388.0,2.849000e-08,2.847000e-08,1.000000e-11
389,389.0,2.854000e-08,2.844000e-08,5.000000e-11
